In [228]:
from collections import Counter

CARD_VALUE_MAP = {
    "A": 14, 
    "K": 13, 
    "Q": 12,
    "J": 11, 
    "T": 10,
}

def card_to_value(card):
    return int(CARD_VALUE_MAP.get(card.label, card.label))

class Card:
    def __init__(self, label):
        self.label = label
        self.value = card_to_value(self)

    def __gt__(self, other):
        return card_to_value(self) > card_to_value(other)

    def __lt__(self, other):
        return card_to_value(self) < card_to_value(other)

    def __eq__(self, other):
        return card_to_value(self) == card_to_value(other)

    def __repr__(self):
        return f"Card('{self.label}')"

class Hand:
    def __init__(self, cards):
        self.cards = cards
        self._counter = Counter((card.label for card in cards))

    def __gt__(self, other):
        if self.hand_value != other.hand_value:
            return self.hand_value > other.hand_value
        
        # Iterate through each hand
        for i, j in zip(self.cards, other.cards):
            if i != j:
                return i.value > j.value
                
        # If all cards are equal, it's a tie
        return False

    def __lt__(self, other):
        if self.hand_value != other.hand_value:
            return self.hand_value < other.hand_value
        
        # Iterate through each hand
        for i, j in zip(self.cards, other.cards):
            if i != j:
                return i.value < j.value
                
        # If all cards are equal, it's a tie
        return False

    def __eq__(self, other):
        return self.cards == other.cards

    def __hash__(self):
        card_labels = [card.label for card in self.cards]
        return hash(''.join(card_labels))

    def __repr__(self):
        cards = ', '.join([repr(card) for card in self.cards])
        return f"Hand([{cards}])"

    @property
    def hand_value(self):
        if self.five_of_a_kind():
            return 6
        elif self.four_of_a_kind():
            return 5
        elif self.full_house():
            return 4
        elif self.three_of_a_kind():
            return 3
        elif self.two_pair():
            return 2
        elif self.one_pair():
            return 1
        elif self.high_card():
            return 0
        else:
            raise RuntimeError("Could not calculate hand value")

    def five_of_a_kind(self):
        values = self._counter.values()
        return len(values) == 1

    def four_of_a_kind(self):
        values = self._counter.values()
        return len(values) == 2 and 4 in values

    def full_house(self):
        values = self._counter.values()
        return len(values) == 2 and 3 in values

    def three_of_a_kind(self):
        values = self._counter.values()
        return len(values) == 3 and 3 in values

    def two_pair(self):
        values = self._counter.values()
        return len(values) == 3 and 2 in values

    def one_pair(self):
        values = self._counter.values()
        return len(values) == 4

    def high_card(self):
        values = self._counter.values()
        return len(values) == 5


In [202]:
test = """
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
"""

def parse_line(line):
    cards, bid = line.split(" ")
    hand = Hand([Card(card) for card in cards])
    return hand, int(bid)

# hand_map = {parse_line(line)[0]: parse_line(line)[1] for line in test.strip().splitlines()}

file_path = 'day_7_input.txt'
with open(file_path, 'r') as file:
    hand_map = {parse_line(line)[0]: parse_line(line)[1] for line in file}

sorted_keys = sorted(hand_map.keys())
bid_rank_map = {hand_map[key]: index for index, key in enumerate(sorted_keys, 1)}
total = sum(bid * rank for bid, rank in bid_rank_map.items())
total

247815719

In [225]:
CARD_VALUE_MAP = {
    "A": 14, 
    "K": 13, 
    "Q": 12,
    "J": 0, 
    "T": 10,
}

class JokerHand:
    def __init__(self, cards):
        self.cards = cards
        self._counter = self.build_counter()

    def __gt__(self, other):
        if self.hand_value != other.hand_value:
            return self.hand_value > other.hand_value
        
        # Iterate through each hand
        for i, j in zip(self.cards, other.cards):
            if i != j:
                return i.value > j.value
                
        # If all cards are equal, it's a tie
        return False

    def __lt__(self, other):
        if self.hand_value != other.hand_value:
            return self.hand_value < other.hand_value
        
        # Iterate through each hand
        for i, j in zip(self.cards, other.cards):
            if i != j:
                return i.value < j.value
                
        # If all cards are equal, it's a tie
        return False

    def __eq__(self, other):
        return self.cards == other.cards

    def __hash__(self):
        card_labels = [card.label for card in self.cards]
        return hash(''.join(card_labels))

    def __repr__(self):
        cards = ', '.join([repr(card) for card in self.cards])
        return f"Hand([{cards}])"

    def build_counter(self):
        counter = Counter((card.label for card in self.cards))
        j_count = counter['J']
        if j_count == 0 or j_count == 5:
            return counter
        else:
            most_common = [key for key, _ in counter.most_common() if key != 'J']
            most_common_card = most_common[0][0]
            # It's not enough to set the J count to zero because we use presence of keys
            # to determine hand value. Therefore we must remove it entirely.
            del counter['J']
            counter[most_common_card] += j_count
            return counter

    @property
    def hand_value(self):
        if self.five_of_a_kind():
            return 6
        elif self.four_of_a_kind():
            return 5
        elif self.full_house():
            return 4
        elif self.three_of_a_kind():
            return 3
        elif self.two_pair():
            return 2
        elif self.one_pair():
            return 1
        elif self.high_card():
            return 0
        else:
            raise RuntimeError("Could not calculate hand value")

    def five_of_a_kind(self):
        values = self._counter.values()
        return len(values) == 1

    def four_of_a_kind(self):
        values = self._counter.values()
        return len(values) == 2 and 4 in values

    def full_house(self):
        values = self._counter.values()
        return len(values) == 2 and 3 in values

    def three_of_a_kind(self):
        values = self._counter.values()
        return len(values) == 3 and 3 in values

    def two_pair(self):
        values = self._counter.values()
        return len(values) == 3 and 2 in values

    def one_pair(self):
        values = self._counter.values()
        return len(values) == 4

    def high_card(self):
        values = self._counter.values()
        return len(values) == 5


In [227]:
test = """
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
"""

def parse_line(line):
    cards, bid = line.split(" ")
    hand = JokerHand([Card(card) for card in cards])
    return hand, int(bid)

# hand_map = {parse_line(line)[0]: parse_line(line)[1] for line in test.strip().splitlines()}

file_path = 'day_7_input.txt'
with open(file_path, 'r') as file:
    hand_map = {parse_line(line)[0]: parse_line(line)[1] for line in file}

sorted_keys = sorted(hand_map.keys())
bid_rank_map = {hand_map[key]: index for index, key in enumerate(sorted_keys, 1)}
total = sum(bid * rank for bid, rank in bid_rank_map.items())
total

248747492

In [229]:
# Five of a kind, where all five cards have the same label: AAAAA
# Four of a kind, where four cards have the same label and one card has a different label: AA8AA
# Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
# Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
# Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
# One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
# High card, where all cards' labels are distinct: 23456

five_of_a_kind = Hand([Card('A'), Card('A'), Card('A'), Card('A'), Card('A')])
four_of_a_kind = Hand([Card('A'), Card('A'), Card('A'), Card('A'), Card('K')])
full_house = Hand([Card('A'), Card('A'), Card('K'), Card('A'), Card('K')])
three_of_a_kind = Hand([Card('A'), Card('A'), Card('J'), Card('A'), Card('K')])
two_pair = Hand([Card('A'), Card('A'), Card('J'), Card('K'), Card('K')])
one_pair = Hand([Card('A'), Card('A'), Card('J'), Card('T'), Card('K')])
high_card = Hand([Card('A'), Card('Q'), Card('J'), Card('T'), Card('K')])

print(five_of_a_kind.hand_value)
print(four_of_a_kind.hand_value)
print(full_house.hand_value)
print(three_of_a_kind.hand_value)
print(two_pair.hand_value)
print(one_pair.hand_value)
print(high_card.hand_value)

6
5
4
3
2
1
0


In [230]:
five_of_a_kind = JokerHand([Card('A'), Card('A'), Card('A'), Card('A'), Card('A')]) # 6
four_of_a_kind = JokerHand([Card('A'), Card('A'), Card('A'), Card('A'), Card('K')]) # 5
full_house = JokerHand([Card('A'), Card('A'), Card('K'), Card('A'), Card('K')]) # 4
three_of_a_kind = JokerHand([Card('A'), Card('A'), Card('J'), Card('A'), Card('K')]) # 5
two_pair = JokerHand([Card('A'), Card('A'), Card('J'), Card('K'), Card('K')]) # 4
one_pair = JokerHand([Card('A'), Card('A'), Card('J'), Card('T'), Card('K')]) # 3
high_card = JokerHand([Card('A'), Card('Q'), Card('J'), Card('T'), Card('K')]) # 1

print(five_of_a_kind.hand_value)
print(four_of_a_kind.hand_value)
print(full_house.hand_value)
print(three_of_a_kind.hand_value)
print(two_pair.hand_value)
print(one_pair.hand_value)
print(high_card.hand_value)

6
5
4
5
4
3
1
